In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [2]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True)

indep_X=df2.drop('classification_yes',axis= 1)
dep_Y=df2['classification_yes']

In [3]:
def fwdselection(indep_X,dep_Y,n):
        rfelist=[]
        
        log_model = LogisticRegression(solver='saga')
        RF = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
       # NB = GaussianNB()
        DT= DecisionTreeClassifier(criterion = 'gini', max_features='sqrt',splitter='best',random_state = 0)
        #svc_model = SVC(kernel = 'linear', random_state = 0)
        #knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        rfemodellist=[log_model,RF,DT] 
        for i in   rfemodellist:
            print(i)
            sfs1= SequentialFeatureSelector(i,n_features_to_select=n,direction='forward',scoring='accuracy')
            sfsfit=sfs1.fit(indep_X,dep_Y)
            featurename= sfsfit.get_support(indices=True)
            selected_features=df2.columns[featurename]
            #Converting index panads to alist
            features=selected_features.to_list()
            # summarize scores       
            print("The features are",features)
            sfs_feature=sfsfit.transform(indep_X)
            rfelist.append(sfs_feature)
        return rfelist

In [13]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    
 
def cm_prediction(classifier,X_test):
     y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
     from sklearn.metrics import confusion_matrix
     cm = confusion_matrix(y_test, y_pred)
        
     from sklearn.metrics import accuracy_score 
     from sklearn.metrics import classification_report 
        #from sklearn.metrics import confusion_matrix
        #cm = confusion_matrix(y_test, y_pred)
        
     Accuracy=accuracy_score(y_test, y_pred )
        
     report=classification_report(y_test, y_pred)
     return  classifier,Accuracy,report,X_test,y_test,cm

def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      
    
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
   
def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm         
    
    
def knn(X_train,y_train,X_test):
           
        # Fitting K-NN to the Training set
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      


def random(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def sfs_Classification(acclog,accknn,accnav,accdes,accrf): 
    
    sfsdataframe=pd.DataFrame(index=['Logistic','Random','DecisionTree'],columns=['Logistic','KNN','Navie','Decision','Random'])
    for number,idex in enumerate(sfsdataframe.index):      
        sfsdataframe['Logistic'][idex]=acclog[number]       
        sfsdataframe['KNN'][idex]=accknn[number]
        sfsdataframe['Navie'][idex]=accnav[number]
        sfsdataframe['Decision'][idex]=accdes[number]
        sfsdataframe['Random'][idex]=accrf[number]
    return sfsdataframe

In [5]:
rfelist=fwdselection(indep_X,dep_Y,5)
rfelist

LogisticRegression(solver='saga')


C:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\a

The features are ['su', 'hrmo', 'sg_d', 'sg_e', 'pc_normal']
RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)
The features are ['al', 'pcv', 'sg_c', 'sg_d', 'dm_yes']
DecisionTreeClassifier(max_features='sqrt', random_state=0)
The features are ['al', 'su', 'hrmo', 'sg_d', 'sg_e']


[array([[ 0.        , 12.51815562,  0.        ,  0.        ,  0.        ],
        [ 0.        , 10.7       ,  0.        ,  0.        ,  1.        ],
        [ 0.        , 12.        ,  0.        ,  0.        ,  1.        ],
        ...,
        [ 0.        ,  9.1       ,  0.        ,  0.        ,  1.        ],
        [ 0.        ,  8.5       ,  0.        ,  0.        ,  1.        ],
        [ 0.        , 16.3       ,  0.        ,  0.        ,  1.        ]]),
 array([[ 3.        , 38.86890244,  1.        ,  0.        ,  0.        ],
        [ 2.        , 34.        ,  1.        ,  0.        ,  0.        ],
        [ 1.        , 34.        ,  0.        ,  0.        ,  0.        ],
        ...,
        [ 3.        , 26.        ,  1.        ,  0.        ,  1.        ],
        [ 0.        , 38.86890244,  0.        ,  0.        ,  1.        ],
        [ 0.        , 53.        ,  0.        ,  0.        ,  0.        ]]),
 array([[ 3.        ,  0.        , 12.51815562,  0.        ,  0.      

In [6]:
acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]

In [7]:
for i in rfelist:   
    X_train, X_test, y_train, y_test=split_scalar(i,dep_Y)   
    
        
    classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
    acclog.append(Accuracy)
    
    #classifier,Accuracy,report,X_test,y_test,cm=svm_linear(X_train,y_train,X_test)  
    #accsvml.append(Accuracy)
    
    #classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
    #accsvmnl.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=knn(X_train,y_train,X_test)  
    accknn.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=Navie(X_train,y_train,X_test)  
    accnav.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)  
    accdes.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
    accrf.append(Accuracy)

In [9]:
sfsdataframe=pd.DataFrame(index=['Logistic','Random','DecisionTree'],columns=['Logistic','KNN','Navie','Decision','Random'])
for number,index in enumerate(sfsdataframe.index):
    sfsdataframe.loc[index,'Logistic']=acclog[number]       
    sfsdataframe.loc[index,'KNN']=accknn[number]
    sfsdataframe.loc[index,'Navie']=accnav[number]
    sfsdataframe.loc[index,'Decision']=accdes[number]
    sfsdataframe.loc[index,'Random']=accrf[number]

In [10]:
sfsdataframe5

,Logistic,KNN,Navie,Decision,Random
Logistic,0.96,0.98,0.76,0.98,0.95
Random,0.96,0.99,0.96,0.99,0.99
DecisionTree,0.99,0.99,0.88,0.99,0.99


In [16]:
rfelist=fwdselection(indep_X,dep_Y,6)
rfelist
acclog=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]

LogisticRegression(solver='saga')


C:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\a

The features are ['su', 'hrmo', 'sg_d', 'sg_e', 'pc_normal', 'ba_present']
RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)
The features are ['al', 'bu', 'pcv', 'sg_c', 'sg_d', 'dm_yes']
DecisionTreeClassifier(max_features='sqrt', random_state=0)
The features are ['al', 'su', 'hrmo', 'sg_c', 'sg_d', 'sg_e']


In [23]:
import warnings
warnings.filterwarnings('ignore')

In [24]:
for i in rfelist:   
    X_train, X_test, y_train, y_test=split_scalar(i,dep_Y)   
    
        
    classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
    acclog.append(Accuracy)
    
    #classifier,Accuracy,report,X_test,y_test,cm=svm_linear(X_train,y_train,X_test)  
    #accsvml.append(Accuracy)
    
    #classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
    #accsvmnl.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=knn(X_train,y_train,X_test)  
    accknn.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=Navie(X_train,y_train,X_test)  
    accnav.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)  
    accdes.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
    accrf.append(Accuracy)

sfsdataframe6=sfs_Classification(acclog,accknn,accnav,accdes,accrf)



In [25]:
sfsdataframe6

,Logistic,KNN,Navie,Decision,Random
Logistic,0.96,0.98,0.76,0.98,0.95
Random,0.97,0.98,0.96,0.99,0.99
DecisionTree,0.98,0.99,0.91,0.99,0.97
